<dl>
<table width="420px">
  <tr>
    <td width="52px"><img src="./kernels/h2opy/logo-64x64.png" alt="H2O R" style="width:48px;height:48px;"></td>
    <td bgcolor="#EFEEEC"><h1><font color="#EFEEEC">__</font>  Python on H<sub>2</sub>O Hadoop</h1></td>
  </tr>
</table>
</dl>
**************

[H<sub>2</sub>O](http://h2o.ai/) is an open source predictive analytics application 


H<sub>2</sub>O is the open source in memory solution from [0xdata](http://h2o-release.s3.amazonaws.com/h2o/rel-slater/5/index.html) for predictive analytics. It enablables distributed processes for solving models. This is done abstractly in what is called an H2O cloud. This cloud can be run in CDH YARN MR2. When run within YARN, the configuration is called H<sub>2</sub>O Hadoop. Further, we can interact with the H<sub>2</sub>O Cloud using a python H<sub>2</sub>O library. In this case the configuration is called Python H<sub>2</sub>O Hadoop. We will demonstrate how to run a simple analyses below:

# Create an H<sub>2</sub>O on Hadoop Application
We'll initiate the hadoop job via an initialization script provided with the demo. It'll start a single node application. It'll take upto 30 seconds to initialize in the quickstart environement.

In [1]:
execfile('../init/h2o_init.py')

Initiating H2O Cloud...


H2O cluster uptime:,10 seconds 166 milliseconds
H2O cluster version:,3.2.0.5
H2O cluster name:,h2o_cloud
H2O cluster total nodes:,1
H2O cluster total memory:,982.0 MB
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321


**************
# Verify H<sub>2</sub>O-On-YARN Application is Running:
Check the status of your application, named **h2o_cloud**, within the [YARN RUNNING Applications](http://localhost:8088/cluster/apps/RUNNING) List. You can also confirm via [H<sub>2</sub>O Flow](http://localhost:54321/).

**************
# Create Frame Using H<sub>2</sub>O  Cluster Connection

While you can import data into the H<sub>2</sub>O cluster using [H<sub>2</sub>O Flow](http://localhost:54321/), we will demonstrate how to train a model within this python session. As of this writing, there is no functionlity to interact with Hive as we had done the spark demonstrations. However, the underlying data in Hive is in HDFS, and H<sub>2</sub>O does allowing importing files directly from HDFS.

We will use an existing source data folder of our Hive table, dat. We generated and loaded this data from script; we know it has the following properties:

$$ y = \beta_0 + \sum_{i=1}^{3} \beta_i x_i$$
$$ \beta_0: 10 $$
$$ \beta_1:  3 $$
$$ \beta_2: -2 $$
$$ \beta_3: -1 $$

In [2]:
dat = h2o.import_file(path="hdfs://quickstart.cloudera:8020/user/cloudera/dat",
                      destination_frame="dat_py",
                      col_names = ["y","x1","x2","x3"])


Parse Progress: [##################################################] 100%
Imported hdfs://quickstart.cloudera:8020/user/cloudera/dat. Parsed 10,000 rows and 4 cols


# Train Model on Frame
With our data imported into the H<sub>2</sub>O cluster, we can train our data using *h2o.glm*. There are quite a few parameters available, so be sure to check out the [H<sub>2</sub>O python module documenation](http://h2o-release.s3.amazonaws.com/h2o/rel-slater/5/docs-website/h2o-py/docs/h2o.html?highlight=h2o.h2o.glm#h2o.h2o.glm) for further details.

In [3]:
model = h2o.glm(y=dat['y'],
                x=dat.drop('y'),
                model_id='GLM_model_py',
                standardize=False,
                intercept=True,
                Lambda=0,
                alpha=0,
                family="gaussian")
print model.__dict__.get('_model_json').get('output').get('coefficients_table')


glm Model Build Progress: [##################################################] 100%
       names  coefficients
0  Intercept     10.000198
1         x1      3.001386
2         x2     -2.000574
3         x3     -1.002177



If everything ran as expected, we should see model weights that estimate the true weights used to generate our dataset:

|         | names     | coefficients |
| ------- | -------------:|:--------------:|
| 0       | Intercept | 10.0 |
| 1       |        x1 |  3.0 |
| 2       |        x2 | -2.0 |
| 3       |        x3 | -1.0 |

# Stop H<sub>2</sub>O-On-YARN Application:
If you were to close out of this python session, the H<sub>2</sub>O cluster would continue to run. To terminate the H<sub>2</sub>O cluster you can shut it down within [H<sub>2</sub>O Flow](http://localhost:54321/) or terminate within the python session as below:

In [ ]:
h2o.shutdown(prompt=False)

# Verify H<sub>2</sub>O-On-YARN Application is Finished:
Check the status of your application, named **h2o_cloud**, within the [YARN All Applications](http://localhost:8088/cluster) List.